In [1]:
import os
from pathlib import Path
import logging
import pandas as pd
import json

os.environ["GARPOS_DIR"] = ("./dev/garpos")
path_cmd = "/Users/gottlieb/.PRIDE_PPPAR_BIN"
os.environ["PATH"] += os.pathsep + path_cmd

from es_sfgtools.pipeline import DataHandler,DATA_TYPE,FILE_TYPE,DataCatalog
#from es_sfgtools.garpos_tools import functions as garpos_functions
#from es_sfgtools.garpos_tools import schemas as garpos_schemas
from es_sfgtools.utils.gage_data import (
    list_survey_files
    )

/Users/gottlieb/working/GIT/es_sfgtools/src/es_sfgtools/pipeline/temp.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Step 1. Initial Setup


#### Browse available surveys from the community archive and select target
- Locate the survey of interest in https://gage-data.earthscope.org/archive/seafloor, and note the `network`, `station`, and `survey` names, which will be input in the cell below.
- In order to use this notebook to process new surveys, the data must first be submitted and made available from the community archive 

In [2]:
network='aleutian'
site='IVB1'
survey='2018_A_SFG1'

#### USE THE FOLLOWING DEFAULTS UNLESS DESIRED####

# Set data directory
data_dir = Path(f"{os.getcwd()}/data/{network}_{site}_{survey}")

# Set Logger Location
logging.basicConfig(level=logging.INFO, 
                    #format="{asctime} - {levelname} - {message}",
                    format="{message}",
                    style="{",
                    datefmt="%Y-%m-%d %H:%M:%S")

# Initialize the data_handler
data_handler = DataHandler(working_dir=data_dir)



Data Handler initialized, data will be stored in /Users/gottlieb/working/GIT/seafloor_geodesy_notebooks/notebooks/data/aleutian_IVB1_2018_A_SFG1


# Step 2. Inventory available data and its location

In [3]:
# Generate a list of files available from remote archive
#TODO: implement options for raw vs intermediate vs processed 
remote_filepaths = list_survey_files(network=network, station=site, survey=survey)

Found under https://gage-data.earthscope.org/archive/seafloor/aleutian/IVB1/2018_A_SFG1/raw:
    1 master file(s)
    73 novatel file(s)
    73 offload file(s)
    73 sonardyne file(s)
    1 lever_arms file(s)
    2 ctd file(s)


In [4]:
# See what files exist locally
data_type_counts = data_handler.get_local_counts()
print(f"Local data directory contains the following:")
for item in data_type_counts.items():
    print(f"    {item[0]}: {item[1]}")

Local data directory contains the following:
    novatel: 73
    sonardyne: 73
    imu: 73
    acoustic: 73
    master: 1
    svpavg: 1


## Step 3. Pull data from remote archive

In [5]:
# TODO: Detail counts of files local vs only remote
#Add found remote files to the catalog
data_handler.add_campaign_data(network=network, station=site, survey=survey, remote_filepaths=remote_filepaths)
data_type_counts = data_handler.get_dtype_counts()
print(f"Catalog now contains the following:")
for item in data_type_counts.items():
    print(f"    {item[0]}: {item[1]}")

File type not recognized for https://gage-data.earthscope.org/archive/seafloor/aleutian/IVB1/2018_A_SFG1/raw/ctd/CTD_SITE2_Ch_Mi


Catalog now contains the following:
    novatel: 73
    offload: 73
    sonardyne: 73
    master: 1
    leverarm: 1
    svpavg: 1


#### Select files types for downloading
Observable file types depend on whether data was collected with an SV2 or SV3 waveglider.  

![Alt text](garpos_flow.jpg)

In [6]:
# Download the files by type
data_handler.download_campaign_data(network=network, station=site, survey=survey, file_type='sonardyne')
data_handler.download_campaign_data(network=network, station=site, survey=survey, file_type='novatel')
data_handler.download_campaign_data(network=network, station=site, survey=survey, file_type='master')
data_handler.download_campaign_data(network=network, station=site, survey=survey, file_type='svpavg')

Data directory currently contains 73 files of type sonardyne
No files downloaded
Data directory currently contains 73 files of type novatel
No files downloaded
Data directory currently contains 1 files of type master
No files downloaded
Data directory currently contains 1 files of type svpavg
No files downloaded


# Step 4. Parse/Process raw data to processing input schemas

- 4.1 Parse acoustic observations into AcousticDataFrames
- 4.2 Parse IMU observations into IMUDataFrames
- 4.3 Process GNSS observables to generate PositionDataFrames
    - Parse RANGE-A novatel messages, build RINEX files
    - Run PRIDE-PPP-AR on RINEX, generate Kin files
    - Parse Kin files into PositionDataFrames
- 4.4 Parse metadata files into SiteConfig

### 4.1 Take all acoustic parent files and generate acoustic df's

In [7]:
data_handler.process_acoustic_data(network=network, station=site, survey=survey)

No unprocessed data found in catalog for types ['dfpo00']
All available instances of processing type dfpo00 to type acoustic have been processed
No unprocessed data found in catalog for types ['sonardyne']
All available instances of processing type sonardyne to type acoustic have been processed


No unprocessed data found in catalog for types ['dfpo00']
All available instances of processing type dfpo00 to type acoustic have been processed
No unprocessed data found in catalog for types ['sonardyne']
All available instances of processing type sonardyne to type acoustic have been processed


### 4.2 Take all IMU parent files and generate IMU df's

In [8]:
data_handler.process_imu_data(network=network, station=site, survey=survey)

No unprocessed data found in catalog for types ['dfpo00']
All available instances of processing type dfpo00 to type imu have been processed
All available instances of processing type novatel to type imu have been processed


No unprocessed data found in catalog for types ['dfpo00']
All available instances of processing type dfpo00 to type imu have been processed
All available instances of processing type novatel to type imu have been processed


### 4.3 Take all GNSS parent files and generate GNSS df's

In [ ]:
data_handler.process_gnss_data(network=network, station=site, survey=survey)

### 4.4 Take all site parent files and generate the site configuration data, which includes ATD offset, sound-velocity DF, and transponder info

In [ ]:
data_handler.process_siteconfig(network=network, station=site, survey=survey)

## Step 5 Merge sessions for GARPOS (each set of temporally identical IMU,Acoustic, and GNSS data frames)

### 5.1 Get a df of all session data

In [ ]:
session_df = data_handler.get_observation_session_data(network=network, station=site, survey=survey,plot=True)
session_df.head()

### 5.2 Generate a dict. of all data corresponding to the temporal groups

In [ ]:

df_paths = data_handler.group_observation_session_data(session_df,timespan='DAY')
print(json.dumps(df_paths, indent=4))

### 5.3 Get Site, ATDOffset, and SVP data

In [ ]:
site_config = data_handler.get_site_config(network=network, station=site, survey=survey)
site_config.name = network + "_" + site + "_" + survey
atd_offset = data_handler.get_atd_offset(network=network, station=site, survey=survey)
svp_data:pd.DataFrame = data_handler.get_svp_data(network=network, station=site, survey=survey)
# Get the first day of data
observables = df_paths["2018-05-14 00:00:00"]


### 5.4 Merge imu, acoustic, and gnss data

In [ ]:
merged_imu = pd.concat([pd.read_csv(path) for path in observables["imu"]])
merged_acoustic = pd.concat([pd.read_csv(path) for path in observables["acoustic"]])
merged_gnss = pd.concat([pd.read_csv(path) for path in observables["gnss"]])


In [ ]:
merged_gnss.head()

In [ ]:
merged_imu.head()


In [ ]:
merged_acoustic.head()

## Step 6 Convert generic data into GARPOS specific formats

In [ ]:

garpos_input = garpos_functions.garpos_input_from_site_obs(
    site_config=site_config,
    atd_offset=atd_offset,
    sound_velocity=svp_data,
    imu_data=merged_imu,
    acoustic_data=merged_acoustic,
    gnss_data=merged_gnss,
)
garpos_fixed = garpos_schemas.GarposFixed()



In [ ]:
for transponder in site_config.transponders:
    print(transponder.position_llh)

## Visualising the data

In [ ]:
data_handler.plot_site_config(site_config=site_config, zoom=7)

In [ ]:
garpos_functions.plot_enu_llh_side_by_side(garpos_input=garpos_input)